In [31]:
# This note is the implementation of past research regarding color metrics

from skimage import util
import collections
import math
import numpy as np
import base64
from PIL import Image
from io import BytesIO
from operator import itemgetter
import matplotlib.image as mpimg
import csv

In [32]:
infiles = [
#'999*ted.com',
'0*theboneandjointcenter.com',
#'1*mozilla.org',
'2*two-n.com',
#'3*segmentfault.com',
#'4*makeitok.org',
'5*disney.co.jp',
'6*wikipedia.org',
'7*news.yahoo.co.jp',
'8*huxiu.com',
#'9*fangdd.com',
'10*cheshi.com',
'11*humblebundle.com',
'12*theatlantic.com',
#'13*superprof.fr',
'14*microsoft.com',
'15*opera.com',
'16*labinthewild.org',
#'17*nounplus.net',
'18*richyli.com',
'19*pxtoem.com', 
#'20*goofy.photo',
'21*javadrive.jp',
#'22*jcodecraeer.com',
#'23*hdpfans.com',
'24*jiqimao.tv',
'25*clamav.net',
'26*bootcdn.cn',
'27*runoob.com',
'28*tensorfly.cn',
'29*journaldugeek.com', 
'30*matetranslate.com',
'31*kameisyouten.ocnk.net',
'32*gingerweb.jp',
'33*cp.pocky.jp',
'34*aladdin-aic.com',
#'35*ho-ginza.net',
#'36*pandayori.com',
#'37*sekimoto.dental',
'38*kokage-m.com',
#'39*coming-saji.com',
#'40*bluewood.bitter.jp',
'41*steakland.jp',
'42*showroomprive.com',
'43*imas-cg.net',
'44*filetender.com',
'45*hexo.io',
'46*yinwang.org',
'47*blog.yitianshijie.net',
'48*yatani.jp',
'49*qiita.com',
'50*52nlp.cn',
'51*guidetojapanese.org',
'52*olderadults.mobi',
'53*blog.whatsapp.com',
#'54*stratechery.com',
'55*lomake.fi',
#'56*theclinic.cl',
'57*bgmaimuna.com',
'58*0dt.net',
'59*web.ics.purdue.edu',
'60*canon-foundation.jp',
#'61*docs.opencv.org',
#'62*interaction-design.org',
#'63*jlpt.jp',
'64*blog.sciencenet.cn',
'65*pantone.com',
#'66*sdtech.co.jp',
'67*news.livedoor.com',
'68*gmo.jp',
'69*tokai-tv.com', 
'70*life-is-tech.com',
'71*bloomberg.co.jp',
'72*cerezo.jp',
'73*tech.nikkeibp.co.jp',
'74*jp.techcrunch.com',
'75*capcom.co.jp',
#'76*kenkun-jinja.org',
'77*sankei.com',
'78*tech-jp.co.jp',
'79*tech-camp.in',
'80*hasegawa-heart.com',
'81*techplay.jp',
#'82*ubejinja.or.jp',
#'83*19lou.com',
#'84*jiankang.com',
'85*cps.com.cn',
#'86*hea.cn',
#'87*jfc.or.jp',
'88*trafst.jp',
'89*infoq.com',
#'90*secure.j-bus.co.jp',
'91*jcr.incites.thomsonreuters.com',
'92*dna.fr',
'93*macg.co',
'94*cvpr2018.thecvf.com',
'95*chi2019.acm.org',
'96*swellnet.com',
'97*klex.ru',
#'98*kudago.com',
'99*theborneopost.com'
]

In [33]:
#Has Problem
def static_clusters(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img= np.array(img)
    img = util.img_as_ubyte(img)
    img = img.reshape(-1, 3)
    img = [tuple(l) for l in img]

    # Get unique colours and their frequencies
    # Divide rgb spectrum (0-255) to a 32x32x32 matrix
    hist = collections.Counter(img)
    hist = hist.items()
    cluster = np.zeros((32, 32, 32))
    hist = list(hist)
    for x in range(len(hist)):
        rc = int(math.ceil((hist[x][0][0] / 8) + 1)) - 1
        gc = int(math.ceil((hist[x][0][1] / 8) + 1)) - 1
        bc = int(math.ceil((hist[x][0][2] / 8) + 1)) - 1
        cluster[rc, gc, bc] += hist[x][1]

    # The amount of cells that have more than 5 entries
    result = (cluster > 5).sum()

    return result

In [34]:
#Has Problem
def dynamic_clusters(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img= np.array(img)
    img = util.img_as_ubyte(img)
    img = img.reshape(-1, 3)
    img = [tuple(l) for l in img]
    hist = collections.Counter(img)
    hist = hist.items()
    frequency = []

    # Create list from histogram
    for x in range(len(hist)):
        add = [hist[x][0][0], hist[x][0][1], hist[x][0][2], hist[x][1]]
        frequency.append(add)

    # Sort the pixels on frequency. This way we can cut the while loop short
    frequency = sorted(frequency, key=itemgetter(3))
    k = len(frequency) - 1

    # Create first cluster
    center_of_clusters = []
    add = [frequency[0][0], frequency[0][1], frequency[0][2], frequency[0][3], 1]
    center_of_clusters.append(add)

    # Find for all colour points a cluster
    while k >= 0:

        # Only colour points with enough presence
        if frequency[k][3] < 6:
            break
        else:
            belong1cluster = False

            # For every colour point calculate distance to all clusters
            for center in range(len(center_of_clusters)):
                point_freq = np.array([frequency[k][0], frequency[k][1], frequency[k][2]])
                point_center = np.array([center_of_clusters[center][0], center_of_clusters[center][1],
                                         center_of_clusters[center][2]])
                distance = np.linalg.norm(point_freq - point_center)

                # If a cluster is close enough, add this colour and recalculate the cluster
                # Now the colour goes to the first cluster fullfilling this. Maybe it should be also the closest?
                if distance <= 3:
                    new_count = center_of_clusters[center][3] + frequency[k][3]
                    new_center = [
                        int((
                                point_freq[0] * frequency[k][3] + point_center[0] * center_of_clusters[center][
                                    3]) / new_count),
                        int((
                                point_freq[1] * frequency[k][3] + point_center[1] * center_of_clusters[center][
                                    3]) / new_count),
                        int((
                                point_freq[2] * frequency[k][3] + point_center[2] * center_of_clusters[center][
                                    3]) / new_count)]
                    center_of_clusters[center][0] = new_center[0]
                    center_of_clusters[center][1] = new_center[1]
                    center_of_clusters[center][2] = new_center[2]
                    center_of_clusters[center][3] = new_count
                    center_of_clusters[center][4] += 1
                    belong1cluster = True
                    break

            # Create new cluster if the colour point is not close enough to other clusters
            if belong1cluster == False:
                add = [frequency[k][0], frequency[k][1], frequency[k][2], frequency[k][3], 1]
                center_of_clusters.append(add)
            k -= 1

    # Only keep clusters with more than 5 colour entries
    new_center_of_clusters = []
    for x in range(len(center_of_clusters)):
        if center_of_clusters[x][4] > 5:
            new_center_of_clusters.append(center_of_clusters[x])

    # Number of clusters, not statistically relevant
    count_dynamic_cluster = len(new_center_of_clusters)

    # Average number of colours per cluster
    average_colour_dynamic_cluster = 0
    for x in range(len(new_center_of_clusters)):
        average_colour_dynamic_cluster += new_center_of_clusters[x][4]

    try:
        average_colour_dynamic_cluster = average_colour_dynamic_cluster / count_dynamic_cluster
    except ZeroDivisionError:
        average_colour_dynamic_cluster = 0

    result = [int(count_dynamic_cluster), int(average_colour_dynamic_cluster)]
    
    return result

In [35]:
def number_of_colours(b64, type=0):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = mpimg.imread(b64, format='PNG')
    img = img[:,:,:3]
    img = img.reshape(-1, 3)
    img = [tuple(l) for l in img]

    # Create histogram
    hist = collections.Counter(img)

    # Count the number of colors: only values that occur more than 5 (for web, type=0) or 2 (for mobile, type=1) times per image are counted
    min_threshold = 5 if type == 0 else 2 if type == 1 else 0
    count_rgb = len({x : hist[x] for x in hist if hist[x] >= min_threshold })

    return count_rgb

In [36]:
def luminance_sd(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img = np.array(img)
    img = util.img_as_ubyte(img)
    img = img.reshape(-1, 3)
    img = [tuple(l) for l in img]

    lum = []
    for pixel in img:
        # Based on: https://en.wikipedia.org/wiki/Luma_(video)
        y = 0.2126 * pixel[0] + 0.7152 * pixel[1] + 0.0722 * pixel[2]
        lum.append(y)

    result = np.std(lum)

    return result

In [37]:
# JPG
static_clusters_list = []
dynamic_clusters_list = []
luminance_sd_list = []

#PNG
number_of_colours_list = []


def main():
    for i in infiles:
        no, title = i.split('*')
#         with open('webpages/'+ title +'.png', 'rb') as f:
        with open('webpagesJ/'+ title +'.jpg', 'rb') as f:
            im = f.read()
            b64 = base64.b64encode(im)
            
        result = luminance_sd(b64)
        
        thisItem = [title, result]
        print(thisItem)
        
        luminance_sd_list.append(thisItem)

    with open('luminance_sd.csv', 'w') as  f:
        writer = csv.writer(f)
        writer.writerow(['title', 'luminance_sd'])
        writer.writerows(luminance_sd_list)

In [38]:
main()

['theboneandjointcenter.com', 71.59877246407491]
['two-n.com', 29.741275635841205]
['disney.co.jp', 82.24280167300662]
['wikipedia.org', 21.698209533506507]
['news.yahoo.co.jp', 83.15630252730175]
['huxiu.com', 70.78385027363741]
['cheshi.com', 75.31011848607478]
['humblebundle.com', 39.012533110686306]
['theatlantic.com', 94.97211095099786]
['microsoft.com', 73.49239780959277]
['opera.com', 51.73310759550712]
['labinthewild.org', 35.44037340868727]
['richyli.com', 36.556025231269594]
['pxtoem.com', 82.55509514891261]
['javadrive.jp', 32.70791159634746]
['jiqimao.tv', 69.08156641167079]
['clamav.net', 103.97832947163815]
['bootcdn.cn', 56.218039055053126]
['runoob.com', 26.655547375199987]
['tensorfly.cn', 42.64061748592886]
['journaldugeek.com', 94.74606394170705]
['matetranslate.com', 44.463845456319966]
['kameisyouten.ocnk.net', 55.07988040290578]
['gingerweb.jp', 69.86022727636713]
['cp.pocky.jp', 93.2143339386526]
['aladdin-aic.com', 44.380127820509614]
['kokage-m.com', 59.9885581